In [64]:
# go wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [65]:
import pysao
from astropy.io import ascii
from astropy.table import Table, Column
import os

In [66]:

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Button, Layout

import numpy as np

In [86]:
def show_next_record():
    global ii, catalog, current_record_idx, info
    records = catalog[ii]
    record = records[current_record_idx]
    #print(record["comment2"])
    
    s = ""
    s = "<style>"
    s = "table, th, td {"
    s = "  border: 1px solid black;"
    s = "  border-collapse: collapse;"
    s = "}"
    s = "th, td {"
    s = "  padding: 20px;"
    s = "}"
    s = "</style>"

    s += "<b>Object {} of {}</b><br>\n".format(current_record_idx+1, len(records))
    
    
    ncols = 3
    tablerows = []
    sr = ""
    z_lya = record["wl_com"] /1216. - 1.
    z_oii = record["wl_com"] /3727. - 1.
    
    s += "<b>z_LyA = {:.2f}</b><br>\n".format(z_lya)
    s += "<b>z_OII = {:.2f}</b><br>\n".format(z_oii)
    
    s += "<table style='width:100%'>\n"
    
    for i,n in enumerate(record.colnames):
        sr += "<td><b>{:10s}:</b> ".format(n)
        sr += str(record[n]) + " "
        if record.columns[n].unit != "":
            sr += str(record.columns[n].unit) + "</td>\n"
        else:
            sr += "</td>\n"
        if i % ncols == 0.:
            tablerows.append(sr)
            sr = ""
    s += "</tr>\n"
    s += "</tr>\n<tr>".join(tablerows)       
    s += "</tr>\n"
    s += "</table>\n"
    
    info.value = s
    comment.value = str(record["comment2"])
    
    if current_record_idx == len(records):
        print("Last objects reached.")
        return
    
    
    id = record["id"]
    x = record["manualx"]
    y = record["manualy"]
    z = record["manualz"]
    if np.isnan(x):
        x = record["x_com"]
    if np.isnan(y):
        y = record["y_com"]
    if np.isnan(z):
        z = record["z_com"]
        
    show_object(x,y,z,vmax,gbimages)
    current_record_idx += 1

def update_catalog(classifier):
    global ii, catalog, current_record_idx, info
    records = catalog[ii]
    
    record = records[current_record_idx-1]
    id = record["id"]
    ifu = record["ifu"]
    jj  = catalog["id"] == id
    jj *= catalog["ifu"] == ifu
    if not np.sum(jj) == 1:
        print("Unable to find object {} for ifu {}.".format(id, ifu))
    catalog["class2"][jj] = classifier
 
def update_comment(comment):
    global ii, catalog, current_record_idx, info
    records = catalog[ii]
    
    record = records[current_record_idx-1]
    id = record["id"]
    ifu = record["ifu"]
    jj  = catalog["id"] == id
    jj *= catalog["ifu"] == ifu
    if not np.sum(jj) == 1:
        print("Unable to find object {} for ifu {}.".format(id, ifu))
    catalog["comment2"][jj] = comment
 

def on_scale_up(b):
    global vmax
    vmax = float(ds9.xpa.get("scale limits").split()[1])
    ds9.xpa.set("frame 1")
    ds9.xpa.set("scale limits 0. {}".format(vmax*1.05))

def on_scale_down(b):
    global vmax
    vmax = float(ds9.xpa.get("scale limits").split()[1])
    ds9.xpa.set("frame 1")
    ds9.xpa.set("scale limits 0. {}".format(vmax*.95))

    
def on_prevslice_clicked(b):
    ds9.xpa.set("frame 1")
    ds9.xpa.set("cube prev")

def on_nextslice_clicked(b):
    ds9.xpa.set("frame 1")
    ds9.xpa.set("cube next")
    
    
def on_prev_clicked(b):
    global current_record_idx
    current_record_idx -=2
    show_next_record()
    #print("Previous object.")

def on_next_clicked(b):
    global current_record_idx, records
    if current_record_idx == len(records):
        print("No more objects.")
        return
    show_next_record()
    
    on_save_clicked(None)
    #print("Next object.")
    
def on_halo_clicked(b):
    update_catalog("halo")
    #show_next_record()
    #print("Selected halo.")
    
def on_fil_clicked(b):
    update_catalog("filament")
    #show_next_record()
    #print("Selected halo.")
    
def on_ps_clicked(b):
    update_catalog("pointsource")
    #show_next_record()
    #print("Selected poinsource.")
    
def on_gal_clicked(b):
    update_catalog("gal")
    #show_next_record()
    #print("Selected poinsource.")
    
def on_agn_clicked(b):
    update_catalog("agn")
    #show_next_record()
    #print("Selected poinsource.")
    

def on_junk_clicked(b):
    update_catalog("junk")
    #show_next_record()
    #print("Selected junk.")
    
    
def on_save_clicked(b):
    global fncatalog

    #catalog.write(newfncatalog, format="ascii.ecsv", overwrite=True)
    
    
    #fncatalog="../data/msf2outcube_{}_allifu.cat".format(field)
    h,t = os.path.split(fncatalog)
    newfncatalog = os.path.join( h , "m" + t)
    catalog.write(newfncatalog, format="ascii.ecsv", overwrite=True)
    
    #print("Wrote {}.".format(newfncatalog))

def on_transfer_contours(b):
    transfer_contours()
    
def on_clear_contours(b):
    clear_contours_on_images()
    

    
def on_add_comment_clicked(b):
    global comment
    update_comment(comment.value)
    

In [73]:
def startup(field, ifu, fncube, fnmap, gbimages, vmax):
    global ds9
    ds9 = pysao.ds9()
    ds9.xpa.set("frame 1")
    ds9.xpa.set("fits {fncube}".format(fncube=fncube))

    ds9.xpa.set("frame lock wcs")
    ds9.xpa.set("cube lock wcs")

    ds9.xpa.set("tile")
    ds9.xpa.set("tile grid mode manual")
    ds9.xpa.set("tile grid layout 5 2")

    ds9.xpa.set("frame 2")
    ds9.xpa.set("fits {fnmap}".format(fnmap=fnmap))
    for i,f in enumerate(gbimages):
        ds9.xpa.set("frame {}".format(i+3))
        ds9.xpa.set("fits {f}".format(f=f))


    ds9.xpa.set("frame 1")
    ds9.xpa.set("cmap staircase")
    ds9.xpa.set("scale limits 0 {}".format(vmax))
    #ds9.xpa.set("crosshair")

    ds9.xpa.set("frame 1")
    ds9.xpa.set("catalog sdss9")
    ds9.xpa.set("frame 1")
    ds9.xpa.set("catalog symbol color white")
    ds9.xpa.set("lock crosshair wcs")
    ds9.xpa.set("catalog close")
    ds9.xpa.set("catalog load ../civano+2016_cls.xml")
    ds9.xpa.set("catalog symbol color green")
    ds9.xpa.set("catalog close")

    for i,f in enumerate(gbimages):
        ds9.xpa.set("frame {}".format(i+3))
        ds9.xpa.set("cmap heat")
        ds9.xpa.set("scale mode 98.")

    regfile="../data/sf2outcube_{field}_{ifu}_photz.reg".format(field=field,ifu=ifu)
    if not os.path.exists(regfile):
        cmd="python ../zphotds9.py {fncube}".format(fncube=fncube)
        !$cmd

    ds9.xpa.set("frame 3")
    #print("regions load {}  color grey".format(regfile))
    ds9.xpa.set("regions load {}".format(regfile))
    ds9.xpa.set("regions color grey ")
    

    ds9.xpa.set("frame 1")
    
    return ds9

showing_contours = False

def clear_contours_on_images():
    global ds9,gbimages, showing_contours
    for i in range(len(gbimages) + 2):
       ds9.xpa.set("frame {}".format(i+1))
       ds9.xpa.set("contour clear ")
        
def show_object(x,y,z,vmax,gbimages):
    global showing_contours

    ds9.xpa.set("frame 1")
    ds9.xpa.set("contour clear ")
    
    if showing_contours:
        clear_contours_on_images()
        showing_contours = False

    ds9.xpa.set("frame 1")
    ds9.xpa.set("pan to {} {} image".format(x,y))
    ds9.xpa.set("crosshair lock wcs")
    ds9.xpa.set("crosshair {} {} image".format(x,y))
    
    ds9.xpa.set("cube {} image".format(int(z)))

    s=vmax/0.6; 
    levels=[ .2*s, .3*s, .4*s, .5*s, .6*s, .8*s, 1.0*s ]
    ds9.xpa.set("frame 1")
    ds9.xpa.set("contour method smooth")
    ds9.xpa.set("contour generate")
    ds9.xpa.set("contour ")
    #print("contour levels " + " ".join([str(l) for l in levels]))
    ds9.xpa.set("contour levels " + " ".join([str(l) for l in levels]))
    ds9.xpa.set("contour color grey")
    
    ds9.xpa.set("contour close ")
    
def transfer_contours():
    global ds9, showing_contours
    
    ds9.xpa.set("frame 1")
    ds9.xpa.set("contour copy")

    for i,f in enumerate(gbimages):
       ds9.xpa.set("frame {}".format(i+3))
       ds9.xpa.set("contour paste wcs white 1 no ")

    #ds9.xpa.set("contours close")
    ds9.xpa.set("frame 1")
    showing_contours = True
    



In [77]:
fncatalog="../data/msf2outcube_{}_allifu.cat".format(field)
h,t = os.path.split(fncatalog)
newfncatalog = os.path.join( h , "m" + t)
catalog.write(newfncatalog, format="ascii.ecsv", overwrite=True)

In [13]:
catalog.write(fncatalog, format="ascii.ecsv", overwrite=True)

In [14]:
vmax = float(ds9.xpa.get("scale limits").split()[1])
ds9.xpa.set("frame 1")
ds9.xpa.set("scale limits 0. {}".format(vmax*.95))

In [89]:
def mkgui():
    global info, comment
    
    button_scaleup = widgets.Button(
        description='scale up',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_scaledown = widgets.Button(
        description='scale down',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )


    button_prevslice = widgets.Button(
        description='previous slice',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_nextslice = widgets.Button(
        description='next slice',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_transfer_contours = widgets.Button(
        description='add contours',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_clear_contours = widgets.Button(
        description='clear contours',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_prev = widgets.Button(
        description='Previous',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_next = widgets.Button(
        description='Next',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_pointsource = widgets.Button(
        description='Pointsource',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_halo = widgets.Button(
        description='Halo',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_fil = widgets.Button(
        description='Filament',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_junk = widgets.Button(
        description='Junk',
        disabled=False,
        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_gal = widgets.Button(
        description='Galaxy',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_agn = widgets.Button(
        description='AGN',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )

    button_save = widgets.Button(
        description='Save',
        disabled=False,
        button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )


    info = widgets.HTML(
        value="Hello <b>World</b>",
        placeholder='Some HTML',
        description='Some HTML',
    )


    comment = widgets.Textarea(
        value='',
        placeholder='Type something',
        description='String:',
        disabled=False
    )

    button_add_comment = widgets.Button(
        description='Add comment',
        disabled=False,
        button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )



    button_prevslice.on_click(on_prevslice_clicked)
    button_nextslice.on_click(on_nextslice_clicked)
    button_scaleup.on_click(on_scale_up)
    button_scaledown.on_click(on_scale_down)
    button_transfer_contours.on_click(on_transfer_contours)
    button_clear_contours.on_click(on_clear_contours)


    button_prev.on_click(on_prev_clicked)
    button_next.on_click(on_next_clicked)
    button_pointsource.on_click(on_ps_clicked)
    button_halo.on_click(on_halo_clicked)
    button_fil.on_click(on_fil_clicked)
    button_gal.on_click(on_gal_clicked)
    button_agn.on_click(on_agn_clicked)
    button_junk.on_click(on_junk_clicked)

    button_save.on_click(on_save_clicked)
    button_add_comment.on_click(on_add_comment_clicked)

    items = [widgets.Label(value="Objects"), button_prev, button_next, \
             button_pointsource, button_halo, button_fil, button_gal, button_agn, button_junk]
    buttonbox1 = widgets.VBox(items, layout=Layout(width='15%', positioning="bottom"))

    items = [widgets.Label(value="ds9 control"), button_prevslice, button_nextslice, button_scaleup, \
             button_scaledown, button_transfer_contours, button_clear_contours,
             widgets.Label(value="catalog"), button_save]
    buttonbox2 = widgets.VBox(items, layout=Layout(width='15%', positioning="bottom"))


    items = [buttonbox1, buttonbox2, info]
    hb1 = widgets.HBox(items)

    items = [comment, button_add_comment]
    hb2 = widgets.HBox(items, layout=Layout(width='100%', positioning="bottom"))

    return widgets.VBox([hb1, hb2])




def load_data(field, ifu, vmax):
    global gbimages, ii, current_record_idx , catalog, records, fncatalog
    fncatalog="../data/msf2outcube_{}_allifu.cat".format(field)


    h,t = os.path.split(fncatalog)
    newfncatalog = os.path.join( h , "m" + t)

    if os.path.exists(newfncatalog):
        print("Reading {}".format(newfncatalog))
        catalog = ascii.read(newfncatalog)
    else:
        print("Reading {}".format(fncatalog))
        catalog = ascii.read(fncatalog)


    B = "../aux/thumbnails/{field}/ifu{ifu}_COSMOS.B.original_psf.v2.fits".format(field=field,ifu=ifu)
    V = "../aux/thumbnails/{field}/ifu{ifu}_COSMOS.V.original_psf.v2.fits".format(field=field,ifu=ifu)
    gp = "../aux/thumbnails/{field}/ifu{ifu}_COSMOS.gp.original_psf.v2.fits".format(field=field,ifu=ifu)
    rp = "../aux/thumbnails/{field}/ifu{ifu}_COSMOS.rp.original_psf.v2.fits".format(field=field,ifu=ifu)
    ip = "../aux/thumbnails/{field}/ifu{ifu}_COSMOS.ip.original_psf.v2.fits".format(field=field,ifu=ifu)
    zp = "../aux/thumbnails/{field}/ifu{ifu}_COSMOS.zp.original_psf.v2.fits".format(field=field,ifu=ifu)
    Ks = "../aux/thumbnails/{field}/ifu{ifu}_COSMOS.Ks.original_psf.v5.fits".format(field=field,ifu=ifu)
    K = "../aux/thumbnails/{field}/ifu{ifu}_COSMOS.K.UV_original_psf.v1.fits".format(field=field,ifu=ifu)

    gbimages=[B,V,gp,rp,ip,zp,Ks,K]

    ii  = catalog["ifu"] == ifu
    ii *= ~np.isnan( catalog["manualx"] )
    current_record_idx = 0
    if not "class2" in catalog.colnames:
        catalog.add_column(Column(["NA"]*len(catalog), name='class2', dtype='S100'), masked=False)
    if not "comment2" in catalog.colnames:
        catalog.add_column(Column([""]*len(catalog), name='comment2', dtype='S500', masked=False))

    catalog = catalog.filled()
    records = catalog[ii]
    ##record = records[0]
    #str(record["comment2"])  
    



field="COSMOSC"
ifu="073"
vmax = 0.6

load_data(field, ifu, vmax)
fncube="../data/sf2outcube_{field}_{ifu}.fits.gz".format(field=field,ifu=ifu)
fnmap="../data/mmap_{field}_{ifu}.fits.gz".format(field=field,ifu=ifu)

! killall ds9
ds9 = startup(field, ifu, fncube, fnmap, gbimages, vmax)

gui = mkgui()

show_next_record()

display(gui)


Reading ../data/mmsf2outcube_COSMOSC_allifu.cat
Reading ../pdz_cosmos2015_v1.3.fits ...
Writing ../data/sf2outcube_COSMOSC_073_photz.reg ...
Wrote ../data/sf2outcube_COSMOSC_073_photz.reg.


In [71]:
"""
../data/COSMOSC_021.txt
../data/COSMOSC_022.txt
../data/COSMOSC_023.txt
../data/COSMOSC_024.txt
../data/COSMOSC_025.txt
../data/COSMOSC_026.txt
../data/COSMOSC_027.txt
../data/COSMOSC_032.txt
../data/COSMOSC_033.txt
../data/COSMOSC_034.txt
../data/COSMOSC_035.txt
../data/COSMOSC_036.txt
../data/COSMOSC_037.txt
../data/COSMOSC_042.txt
../data/COSMOSC_043.txt
../data/COSMOSC_044.txt
../data/COSMOSC_045.txt
../data/COSMOSC_046.txt
../data/COSMOSC_047.txt
../data/COSMOSC_052.txt
../data/COSMOSC_053.txt
../data/COSMOSC_062.txt
../data/COSMOSC_063.txt
../data/COSMOSC_072.txt
../data/COSMOSC_073.txt
../data/COSMOSC_074.txt # done
../data/COSMOSC_075.txt # done
../data/COSMOSC_076.txt # done
../data/COSMOSC_077.txt # done
../data/COSMOSC_082.txt # done
../data/COSMOSC_083.txt # done
../data/COSMOSC_084.txt # done 
../data/COSMOSC_085.txt # done 
../data/COSMOSC_086.txt # done
../data/COSMOSC_087.txt # done
../data/COSMOSC_093.txt # done
../data/COSMOSC_095.txt # done
../data/COSMOSC_096.txt # done
../data/COSMOSC_097.txt # done, very many objects
../data/COSMOSC_103.txt # done
../data/COSMOSC_104.txt # done
../data/COSMOSC_105.txt # done
../data/COSMOSC_106.txt # done
"""

'\n../data/COSMOSC_021.txt\n../data/COSMOSC_022.txt\n../data/COSMOSC_023.txt\n../data/COSMOSC_024.txt\n../data/COSMOSC_025.txt\n../data/COSMOSC_026.txt\n../data/COSMOSC_027.txt\n../data/COSMOSC_032.txt\n../data/COSMOSC_033.txt\n../data/COSMOSC_034.txt\n../data/COSMOSC_035.txt\n../data/COSMOSC_036.txt\n../data/COSMOSC_037.txt\n../data/COSMOSC_042.txt\n../data/COSMOSC_043.txt\n../data/COSMOSC_044.txt\n../data/COSMOSC_045.txt\n../data/COSMOSC_046.txt\n../data/COSMOSC_047.txt\n../data/COSMOSC_052.txt\n../data/COSMOSC_053.txt\n../data/COSMOSC_062.txt\n../data/COSMOSC_063.txt\n../data/COSMOSC_072.txt\n../data/COSMOSC_073.txt\n../data/COSMOSC_074.txt\n../data/COSMOSC_075.txt\n../data/COSMOSC_076.txt\n../data/COSMOSC_077.txt\n../data/COSMOSC_082.txt\n../data/COSMOSC_083.txt\n../data/COSMOSC_084.txt\n../data/COSMOSC_085.txt # done \n../data/COSMOSC_086.txt # done\n../data/COSMOSC_087.txt # done\n../data/COSMOSC_093.txt # done\n../data/COSMOSC_095.txt # done\n../data/COSMOSC_096.txt # done\n../d